In [1]:
-

/home/manager/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/manager/anaconda2/lib/python2.7/lib-tk/Tkinter.py", line 1542, in __call__
    return self.func(*args)
  File "<ipython-input-1-0ac9b389753f>", line 295, in ChangeGraph
    self.RefreshGraph(tipocliente,tiposerie)
  File "<ipython-input-1-0ac9b389753f>", line 243, in RefreshGraph
    self.Wlog('Cambiamento grafici visualizzati in corso ... '+Decl[tiposerie]+' - '+Decl[tipocliente])
KeyError: ''
/home/manager/anaconda2/lib/python2.7/site-packages/statsmodels/tsa/tsatools.py:652: RuntimeWarning: invalid value encountered in double_scalars
  tmp[kiter] = (macoefs[kiter]-b *macoefs[j-kiter-1])/(1-b**2)
/home/manager/anaconda2/lib/python2.7/site-pa

In [ ]:
dfp1 = pd.read_csv('NCFAdati1.csv',sep=';',index_col=0, parse_dates=True, infer_datetime_format=True)
dfp2 = pd.read_csv('NCFAdati2.csv',sep=';',index_col=0)
        

In [ ]:
dfp2.info()

In [ ]:
dfp2.loc['MON']['FORE']

In [ ]:
a = list(set(dfp2['FORE'].tolist()))
b=list(a)
b.append('BDT')
print a
print b